In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import pandas as pd
import time

from bs4 import BeautifulSoup

In [2]:
def flatten(t):
    return [item for sublist in t for item in sublist]

In [3]:
# driver = webdriver.Chrome('C:/Users/user/Documents/chromedriver.exe')
urls= ['https://www.gallito.com.uy/inmuebles/casas/venta/montevideo/ord_asc?pag={}'.format(i) for i in range(1, 3)]

driver = webdriver.Chrome('/home/guillermo/Documentos/chromedriver_linux64/chromedriver')

In [4]:
urls

['https://www.gallito.com.uy/inmuebles/casas/venta/montevideo/ord_asc?pag=1',
 'https://www.gallito.com.uy/inmuebles/casas/venta/montevideo/ord_asc?pag=2']

In [5]:
driver.get('https://www.gallito.com.uy/inmuebles/casas/venta/montevideo/ord_asc?pag=1')

In [6]:
data = []
metraje = []
url_publicacion = []

for url in urls:
    try:
        driver.get(url)
    except:
        print('Get method failed, check URL.')
    pagina = [e.text.splitlines() for e in driver.find_elements_by_css_selector('.contenedor-info')]
    data.append(pagina)
    html = [e.get_attribute("innerHTML") for e in driver.find_elements_by_css_selector('.contenedor-info .mas-info')]
    for i in html:
        soup = BeautifulSoup(i, "html.parser")
        metraje.append(soup.span.text)
        url_publicacion.append(soup.find('a')['href'])
flat_list = flatten(data)
result = pd.DataFrame(flat_list, columns=['desc', 'valor'])
result['metraje'] = metraje
result['metraje'] = result['metraje'].replace({' m2': ''}, regex=True)
result['url'] = url_publicacion

In [9]:
result.head()

,desc,valor,metraje,url
0,2 Dormitorios en Ciudad Vieja,U$S 1,160,https://www.gallito.com.uy/cerrito-y-juan-carl...
1,Más de 4 dormitorios en Pocitos,U$S 1,300,https://www.gallito.com.uy/casa-avenida-brasil...
2,Más de 4 dormitorios en Otro,U$S 100,500,https://www.gallito.com.uy/compro-terreno-o-“c...
3,3 Dormitorios en Maroñas Curva,U$S 115,82,https://www.gallito.com.uy/casa-muy-cuidada-a-...
4,3 Dormitorios en Goes,U$S 120,67,https://www.gallito.com.uy/oportunidad-inversi...


In [69]:
# recupera latlong si hay
coordenadas = []

driver.get('https://www.gallito.com.uy/due%C3%B1o-vende-casa-ph-inmuebles-19426843')

elem = driver.find_elements_by_css_selector('#ubicacion')

if elem != []:
    element = driver.find_element_by_xpath("/html/body/form/main/div/div[1]/ul/li[4]/a/i")
    element.click()
    time.sleep(2)
    ubic_html = [e.get_attribute("innerHTML") for e in driver.find_elements_by_css_selector("#ubicacion")]
    soup = BeautifulSoup(ubic_html[0], "html.parser")
    src = soup.find('iframe')['src']
    latlng = src.split('q=', 1)[1].split('0&zoom=', 1)[0]
    coordenadas.append(latlng)
else:
    print('Sin ubicación')
    coordenadas.append('')


Elem not enabled


In [73]:
urls = list(result.url)

In [ ]:
urls= ['https://www.gallito.com.uy/inmuebles/casas/venta/montevideo/ord_asc?pag={}'.format(i) for i in range(1, 155)]

In [ ]:
def scrapper(urls):
    data = []
    metraje = []
    for url in urls:
        try:
            driver.get(url)
        except:
            print('Get method failed, check URL.')
        pagina = [e.text.splitlines() for e in driver.find_elements_by_css_selector('.contenedor-info')]
        data.append(pagina)
        html = [e.get_attribute("innerHTML") for e in driver.find_elements_by_css_selector('.contenedor-info .mas-info')]

        for i in html:
            soup = BeautifulSoup(i, "html.parser")
            metraje.append(soup.span.text)

        time.sleep(1)

    flat_list = flatten(data)
    result = pd.DataFrame(flat_list, columns=['desc', 'valor'])
    result['metraje'] = metraje
    result['metraje'] = result['metraje'].replace({' m2': ''}, regex=True)
    
    return result

In [ ]:
data = scrapper(urls)

In [ ]:
data.head(10)

In [ ]:
data.shape

In [ ]:
data.to_csv('ventas_mvdeo_20210706.csv', index=False)

In [ ]:
# limpieza
data = pd.read_csv('ventas_mvdeo_20210706.csv')

data['valor'].str.replace('U', '')

for word, rep in {"U":" ", "S":"", "$":"", ".":""}.items():
    data['valor'] = data['valor'].str.replace(word, rep, regex=False)

In [ ]:
data['valor'] = data['valor'].astype(int)

In [ ]:
len(data)

In [ ]:
data = data.loc[~((data.valor == 111111111) | (data.valor < 45000))]
data.head()

In [ ]:
data[['dormitorios', 'barrio']] = data['desc'].str.split(' en ', n = 1, expand = True)

In [ ]:
# data.to_csv('ventas_mvdeo_20210706_depu.csv', index=False)

In [ ]:
# pd.Series(data.barrio.unique()).to_csv('cod.csv')

In [ ]:
data['barrio_ine'] = data['barrio']

In [ ]:
def format_barrio(df, column):
    "Formatea strings de Barrio Gallito a Barrio INE"
    barrios_dict = {
        'Aguada' : 'Aguada',
        'Aires Puros' : 'Aires Puros',
        'Atahualpa' : 'Atahualpa',
        'B. De Carrasco' : 'Bañados de Carrasco',
        'Barrio Sur' : 'Barrio Sur',
        'Belvedere' : 'Belvedere',
        'Brazo Oriental' : 'Brazo Oriental',
        'Buceo' : 'Buceo',
        'Capurro' : 'Capurro, Bella Vista',
        'Carrasco' : 'Carrasco',
        'Carrasco Norte' : 'Carrasco Norte',
        'Casabo' : 'Casabó, Pajas Blancas',
        'Casavalle' : 'Casavalle',
        'Centro' : 'Centro',
        'Cerrito' : 'Cerrito',
        'Cerro' : 'Cerro',
        'Ciudad Vieja' : 'Ciudad Vieja',
        'Cno. Maldonado' : 'Colón Centro y Noroeste',
        'Colon' : 'Colón Sureste, Abayubá',
        'Cordon' : 'Cordón',
        'Golf' : 'Flor de Maroñas',
        'Ituzaingo' : 'Ituzaingó',
        'J. Hipodromo' : 'Jardines del Hipódromo',
        'Jacinto Vera' : 'Jacinto Vera',
        'La Blanqueada' : 'La Blanqueada',
        'La Comercial' : 'La Comercial',
        'La Figurita' : 'La Figurita',
        'La Teja' : 'La Teja',
        'Larrañaga' : 'Larrañaga',
        'Las Acacias' : 'Las Acacias',
        'Lezica' : 'Lezica, Melilla',
        'Malvin' : 'Malvín',
        'Malvin Norte' : 'Malvín Norte',
        'Manga' : 'Manga',
        'Maroñas' : 'Maroñas, Parque Guaraní',
        'Maroñas Curva' : 'Maroñas, Parque Guaraní',
        'Melilla' : 'Lezica, Melilla',
        'Nuevo Paris' : 'Mercado Modelo, Bolívar',
        'Pajas Blancas' : 'Nuevo París',
        'Palermo' : 'Palermo',
        'Parque Batlle' : 'Parque Batlle, Villa Dolores',
        'Parque Rodo' : 'Parque Rodó',
        'Paso De La Arena' : 'Paso de la Arena',
        'Peñarol' : 'Peñarol, Lavalleja',
        'Perez Castellanos' : 'Castro, P. Castellanos',
        'Piedras Blancas' : 'Piedras Blancas',
        'Pocitos' : 'Pocitos',
        'Pocitos Nuevo' : 'Pocitos',
        'Prado' : 'Prado, Nueva Savona',
        'Prado Norte' : 'Prado, Nueva Savona',
        'Puerto Buceo' : 'Buceo',
        'Punta Carretas' : 'Punta Carretas',
        'Punta Gorda' : 'Punta Gorda',
        'Punta Rieles' : 'Punta Rieles, Bella Italia',
        'Reducto' : 'Reducto',
        'Sayago' : 'Sayago',
        'Tres Cruces' : 'Tres Cruces',
        'Union' : 'Unión',
        'Villa Dolores' : 'Parque Batlle, Villa Dolores',
        'Villa Española' : 'Villa Española',
        'Villa Muñoz' : 'Villa Muñoz, Retiro'
        }

    return df[column].map(barrios_dict)

In [ ]:
data['barrio_ine'] = format_barrio(data, 'barrio')

In [ ]:
data.head(15)

In [ ]:
def encode_barrio(df, column):
    "Codifica barrio INE"
    barrios_dict = {
    'Ciudad Vieja':1,
    'Centro':2,
    'Barrio Sur':3,
    'Cordón':4,
    'Palermo':5,
    'Parque Rodó':6,
    'Punta Carretas':7,
    'Pocitos':8,
    'Buceo':9,
    'Parque Batlle, Villa Dolores':10,
    'Malvín':11,
    'Malvín Norte':12,
    'Punta Gorda':13,
    'Carrasco':14,
    'Carrasco Norte':15,
    'Bañados de Carrasco':16,
    'Maroñas, Parque Guaraní':17,
    'Flor de Maroñas':18,
    'Las Canteras':19,
    'Punta Rieles, Bella Italia':20,
    'Jardines del Hipódromo':21,
    'Ituzaingó':22,
    'Unión':23,
    'Villa Española':24,
    'Mercado Modelo, Bolívar':25,
    'Castro, P. Castellanos':26,
    'Cerrito':27,
    'Las Acacias':28,
    'Aires Puros':29,
    'Casavalle':30,
    'Piedras Blancas':31,
    'Manga, Toledo Chico':32,
    'Paso de las Duranas':33,
    'Peñarol, Lavalleja':34,
    'Cerro':35,
    'Casabó, Pajas Blancas':36,
    'La Paloma, Tomkinson':37,
    'La Teja':38,
    'Prado, Nueva Savona':39,
    'Capurro, Bella Vista':40,
    'Aguada':41,
    'Reducto':42,
    'Atahualpa':43,
    'Jacinto Vera':44,
    'La Figurita':45,
    'Larrañaga':46,
    'La Blanqueada':47,
    'Villa Muñoz, Retiro':48,
    'La Comercial':49,
    'Tres Cruces':50,
    'Brazo Oriental':51,
    'Sayago':52,
    'Conciliación':53,
    'Belvedere':54,
    'Nuevo París':55,
    'Tres Ombúes, Victoria':56,
    'Paso de la Arena':57,
    'Colón Sureste, Abayubá':58,
    'Colón Centro y Noroeste':59,
    'Lezica, Melilla':60,
    'Villa García, Manga Rural':61,
    'Manga':62
        }
    return df[column].map(barrios_dict)

In [ ]:
data['cod_barrio_ine'] = encode_barrio(data, 'barrio_ine')
data.head()

In [ ]:

data = data.loc[(data.metraje > 10) & (data.metraje < 4000) & (~data.metraje.isna())]

data['metraje'] = data['metraje'].astype(int)

data['valor_metro'] = data['valor'] / data['metraje']

agru = data.groupby('cod_barrio_ine').median('valor_metro').reset_index()

agru['cod_barrio_ine'] = agru['cod_barrio_ine'].astype(int)
                                                       
agru.to_csv('agru.csv', index=False)